## Part 1 - 卷积基本知识

In [4]:
import torch 
from torch import nn

#手动实现一个二维卷积算子，stride为1
def corr2d(X, K):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    h, w = K.shape    
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y


# 注意点乘和叉乘的用法
a = torch.tensor([[1,1],[1,1]])
b = torch.tensor([[1,1],[1,1]])

print(a*b)

print(a.mm(b))

tensor([[1, 1],
        [1, 1]])
tensor([[2, 2],
        [2, 2]])


In [5]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)


tensor([[19., 25.],
        [37., 43.]])

In [17]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn((1,kernel_size)))
        self.bias = nn.Parameter(torch.randn(1))
        print(self.weight.shape)
        print(self.weight)

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


In [18]:
net = nn.Sequential(
        nn.Flatten(),
        Conv2D(2),
        )

net(X)

torch.Size([1, 2])
Parameter containing:
tensor([[-1.8863, -0.3301]], requires_grad=True)


tensor([[  0.6494,  -1.5670],
        [ -5.9997,  -8.2161],
        [-12.6488, -14.8652]], grad_fn=<AddBackward0>)